This is the file were using to test the llama api.

Create Api request object: 

We define the conent a

In [3]:
import pandas as pd
import numpy as np
import seaborn as sn
import json
import matplotlib.pyplot as plt
from collections import Counter
from llamaapi import LlamaAPI
from dotenv import load_dotenv
import os 

Creating the LLAMA object to start using the API

In [5]:
load_dotenv()
# Create a new LlamaAPI object
LLAMA_API_KEY = os.getenv("LLAMA_API_KEY")
llama = LlamaAPI(LLAMA_API_KEY)

We define the content as: What is the sector and industry of this job

We give the API the following infomration: job description, responsibilities, and skills 

In [ ]:
api_request_json = {
  "model": "llama3.1-70b",
  "messages": [
    {"role": "user", "content": "What is the sector and industry of this job?"}, 
  ],
  "functions": [
    {
        "name": "get_sector_industry",
        "description": 
            "Get the sector and industry from this given job information such as skills, responsibilites, and description",
        "parameters": {
            "type": "object",
        "properties": { 
            "job description": {
                "type": "string",
                "description": "The job description for the role"
            },
            "responsibilites": {
                "type": "string",
                "description": "The job responsibilites for the role"
            },
            "skills": {
                "type": "string",
                "description": "The required skills listed for the role"
            },
        },
        "required": ["job description", "responsibilites", "skills"] 
        }
    }
  ],
}

response = llama.run(api_request_json)
output = response.json()['choices'][0]['message']

Function call for retrieving info from online

In [ ]:
def get_sector_industry(location):
# caroline says to not use a specific external url/database to query for info -- this is what the email and weather functions used but our case is different
# Need to create function to get skills, responsibilites, and description from job posting

In [ ]:
email_information = []
def get_emails(quantity, login, password):
    try:
        mail = imaplib.IMAP4_SSL("imap.gmail.com")
        mail.login(login, password)

        mail.select("inbox")

        status, messages = mail.search(None, "(UNSEEN)")

        email_ids = messages[0].split()[::-1]

        limite = min(quantity, len(email_ids))

        for i in range(limite):
            email_id = email_ids[i]
            _, msg_data = mail.fetch(email_id, "(RFC822)")
            for response_part in msg_data:
                if isinstance(response_part, tuple):
                    email_message = email.message_from_bytes(response_part[1])
                    subject, encoding = decode_header(email_message["Subject"])[0]
                    if isinstance(subject, bytes):
                        subject = subject.decode(encoding or "utf-8")

                    sender = email_message.get("From")

            email_information.append({"Subject": subject, "From": sender})
                    

        mail.logout()
        return json.dumps(email_information)
    except Exception as e:
        print("Err:", str(e))